# RNN을 이용한 텍스트 생성(Text Generation using RNN)

## 1. RNN을 이용하여 텍스트 생성하기

'경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장

|samples|X|y|
|---|---|---|
|1|경마장에|있는|
|2|경마장에 있는|말이|
|3|경마장에 있는 말이|뛰고|
|4|경마장에 있는 말이 뛰고|있다|
|5|그의|말이|
|6|그의 말이|법이다|
|7|가는|말이|
|8|가는 말이|고와야|
|9|가는 말이 고와야|오는|
|10|가는 말이 고와야 오는|말이|
|11|가는 말이 고와야 오는 말이|곱다|

### ) 데이터에 대한 이해와 전처리

In [1]:
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
# 단어 집합 생성 및 크기 확인
t = Tokenizer()

t.fit_on_texts( [ text ] )
vocab_size = len( t.word_index ) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 

print( '단어 집합의 크기 : %d' % vocab_size )

단어 집합의 크기 : 12


In [4]:
# 각 단어와 부여된 정수 인덱스
print( t.word_index )

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
# 훈련 데이터
sequences = list()
for line in text.split( '\n' ): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences( [ line ] )[ 0 ]
    for i in range( 1, len( encoded ) ):
        sequence = encoded[ :i + 1 ]
        sequences.append( sequence )

print( '학습에 사용할 샘플의 개수: %d' % len( sequences ) )

학습에 사용할 샘플의 개수: 11


In [6]:
# 전체 샘플
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


- 위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. 
- [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 
- 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

- 우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 
- 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 
- 이를 코드로는 다음과 같이 구할 수 있습니다.

In [7]:
max_len = max( len( l ) for l in sequences ) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print( '샘플의 최대 길이 : {}'.format( max_len ) )

샘플의 최대 길이 : 6


In [8]:
# 전체 샘플의 길이를 6으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

- pad_sequences()는 모든 샘플에 대해서 0을 사용하여 길이를 맞춰줍니다. 
- maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다. 
- 전체 훈련 데이터를 출력해봅니다.

In [9]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


- 길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨습니다. 
- 이제 각 샘플의 마지막 단어를 레이블로 분리합시다. 
- 레이블의 분리는 Numpy를 이용해서 가능합니다.

In [10]:
sequences = np.array( sequences )
X = sequences[ :, :-1 ]
y = sequences[ :, -1 ]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [11]:
print( X )

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [13]:
print( y ) # 모든 샘플에 대한 레이블 출력

[ 3  1  4  5  1  7  1  9 10  1 11]


- 레이블이 분리되었습니다. 
- 이제 RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [14]:
y = to_categorical( y, num_classes = vocab_size )

In [15]:
print( y )

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

각 단어의 임베딩 벡터는 10차원을 가지고, 32의 은닉 상태 크기를 가지는 바닐라 RNN을 사용합니다.

In [17]:
model = Sequential()
model.add( Embedding( vocab_size, 10, input_length = max_len-1 ) ) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add( SimpleRNN( 32 ) )
model.add( Dense( vocab_size, activation = 'softmax' ) )
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )
model.fit( X, y, epochs = 200, verbose = 2 )

Train on 11 samples
Epoch 1/200
11/11 - 3s - loss: 2.4834 - accuracy: 0.0909
Epoch 2/200
11/11 - 0s - loss: 2.4691 - accuracy: 0.0909
Epoch 3/200
11/11 - 0s - loss: 2.4549 - accuracy: 0.4545
Epoch 4/200
11/11 - 0s - loss: 2.4406 - accuracy: 0.4545
Epoch 5/200
11/11 - 0s - loss: 2.4262 - accuracy: 0.4545
Epoch 6/200
11/11 - 0s - loss: 2.4115 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.3965 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.3812 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.3655 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.3493 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.3326 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.3153 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.2974 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2788 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2595 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.2397 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.2192 - accuracy: 0.3636
Epo

Epoch 142/200
11/11 - 0s - loss: 0.3178 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.3117 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.3057 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2998 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2940 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2884 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2828 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2773 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2719 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2666 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2614 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2563 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2513 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2464 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2415 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2368 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2322 - accuracy: 1.00

모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해봅시다.

In [18]:
def sentence_generation( model, t, current_word, n ): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range( n ): # n번 반복
        encoded = t.texts_to_sequences( [ current_word ] )[ 0 ] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences( [ encoded ], maxlen = 5, padding = 'pre') # 데이터에 대한 패딩
        result = model.predict_classes( encoded, verbose = 0 )
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [19]:
print( sentence_generation( model, t, '경마장에', 4 ) )
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

경마장에 있는 말이 뛰고 있다


In [20]:
print( sentence_generation( model, t, '그의', 2 ) ) # 2번 예측

그의 말이 법이다


In [21]:
print( sentence_generation( model, t, '가는', 5 ) ) # 5번 예측

가는 말이 고와야 오는 말이 곱다


- 이제 앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줍니다. 
- 이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4, 2, 5를 각각 인자값으로 주었습니다. 
- 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 합니다.

## 2. LSTM을 이용하여 텍스트 생성하기

### 1) 데이터에 대한 이해와 전처리

사용할 데이터는 뉴욕 타임즈 기사의 제목입니다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드 합니다.

파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [23]:
from string import punctuation

import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [25]:
# 다운로드한 훈련 데이터를 데이터프레임에 저장합니다
df = pd.read_csv( './ArticlesApril2018.csv' )
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [26]:
print( '열의 개수: ', len( df.columns ) )
print( df.columns )

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [28]:
# 사용할 열은 제목에 해당되는 headline 열입니다. Null 값이 있는지 확인해봅시다.
df[ 'headline' ].isnull().values.any()

False

In [30]:
# Null 값은 별도로 없는 것으로 보입니다. headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.
headline = [] # 리스트 선언
headline.extend( list( df.headline.values ) ) # 헤드라인의 값들을 리스트로 저장
headline[ :5 ] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

headline이라는 리스트에 모든 신문 기사의 제목을 저장했습니다. 저장한 리스트에서 상위 5개만 출력해보았습니다.

그런데 4번째, 5번째 샘플에 Unknown 값이 들어가있습니다. headline 전체에 걸쳐서 Unknown 값을 가진 샘플이 있을 것으로 추정됩니다. 비록 Null 값은 아니지만 지금 하고자 하는 실습에 도움이 되지 않는 노이즈 데이터이므로 제거해줄 필요가 있습니다. 제거하기 전에 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해보겠습니다.

In [32]:
print( '총 샘플의 개수 : {}'.format( len( headline ) ) ) # 현재 샘플의 개수

총 샘플의 개수 : 1324


In [34]:
# 노이즈 데이터를 제거하기 전 데이터의 개수는 1,324입니다. 즉, 신문 기사의 제목이 총 1,324개입니다.
headline = [ n for n in headline if n != "Unknown" ] # Unknown 값을 가진 샘플 제거
print( '노이즈값 제거 후 샘플의 개수 : {}'.format( len( headline ) ) )# 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


In [35]:
# 샘플의 수가 1,324에서 1,214로 110개의 샘플이 제거되었는데, 기존에 출력했던 5개의 샘플을 출력해보겠습니다.
headline[ :5 ]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

기존에 4번째, 5번째 샘플에서는 Unknown 값이 있었는데 현재는 제거가 된 것을 확인하였습니다. 이제 데이터 전처리를 수행합니다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다. 전처리를 수행하고, 다시 샘플 5개를 출력합니다.

In [36]:
def repreprocessing( s ):
    s = s.encode( "utf8" ).decode( "ascii", 'ignore' )
    return ''.join( c for c in s if c not in punctuation ).lower() # 구두점 제거와 동시에 소문자화

text = [ repreprocessing( x ) for x in headline ]
text[ :5 ]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

기존의 출력과 비교하면 모든 단어들이 소문자화되었으며 N.F.L.이나 Cheerleaders’ 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거되었습니다. 이제 단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [37]:
t = Tokenizer()

t.fit_on_texts( text )
vocab_size = len( t.word_index ) + 1
print( '단어 집합의 크기 : %d' % vocab_size )

단어 집합의 크기 : 3494


총 3,494개의 단어가 존재합니다. 이제 정수 인코딩과 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성합니다.

In [38]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences( [ line ] )[ 0 ] # 각 샘플에 대한 정수 인코딩
    for i in range( 1, len( encoded ) ):
        sequence = encoded[ :i + 1 ]
        sequences.append( sequence )

sequences[ :11 ] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

이해를 돕기 위해 출력 결과에 주석을 추가하였습니다. 
왜 하나의 문장을 저렇게 나눌까요? 예를 들어 '경마장에 있는 말이 뛰고 있다' 라는 문장 하나가 있을 때, 
최종적으로 원하는 훈련 데이터의 형태는 다음과 같습니다. 하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고하는 것입니다.

|samples|X|y|
|---|---|---|
|1|경마장에|있는|
|2|경마장에 있는|말이|
|3|경마장에 있는 말이|뛰고|
|4|경마장에 있는 말이 뛰고|있다|

위의 sequences는 모든 문장을 각 단어가 각 시점(time step)마다 하나씩 추가적으로 등장하는 형태로 만들기는 했지만, 
아직 예측할 단어에 해당되는 레이블을 분리하는 작업까지는 수행하지 않은 상태입니다. 
어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듭니다.

In [40]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[ value ] = key

print( '빈도수 상위 582번 단어 : {}'.format( index_to_word[ 582 ] ) )

빈도수 상위 582번 단어 : offer


582이라는 인덱스를 가진 단어는 본래 offer라는 단어였습니다. 원한다면 다른 숫자로도 시도해보세요. 이제 y데이터를 분리하기 전에 전체 샘플의 길이를 동일하게 만드는 패딩 작업을 수행합니다. 패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인합니다.

In [41]:
max_len = max( len( l ) for l in sequences )
print( '샘플의 최대 길이 : {}'.format( max_len ) )

샘플의 최대 길이 : 24


가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩하겠습니다.

In [42]:
sequences = pad_sequences( sequences, maxlen = max_len, padding = 'pre' )
print( sequences[ :3 ] )

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


padding='pre'를 설정하여 샘플의 길이가 24보다 짧은 경우에 앞에 0으로 패딩되었습니다. 이제 맨 우측 단어만 레이블로 분리합니다.

In [43]:
sequences = np.array( sequences )
X = sequences[ :, :-1 ]
y = sequences[ :, -1 ]

In [44]:
print( X[ :3 ] )

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


훈련 데이터 X에서 3개의 샘플만 출력해보았는데, 맨 우측에 있던 정수값 269, 371, 1115가 사라진 것을 볼 수 있습니다. 뿐만 아니라, 각 샘플의 길이가 24에서 23으로 줄었습니다.

In [45]:
print( y[ :3 ] ) # 레이블

[ 269  371 1115]


훈련 데이터 y 중 3개의 샘플만 출력해보았는데, 기존 훈련 데이터에서 맨 우측에 있던 정수들이 별도로 저장되었습니다.

In [46]:
y = to_categorical( y, num_classes = vocab_size )

레이블 데이터 y에 대해서 원-핫 인코딩을 수행하였습니다. 이제 모델을 설계합니다.

### 2) 모델 설계하기

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [48]:
model = Sequential()
model.add( Embedding( vocab_size, 10, input_length = max_len - 1 ) )
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add( LSTM( 128 ) )
model.add( Dense( vocab_size, activation = 'softmax' ) )

model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )
model.fit( X, y, epochs = 200, verbose = 2 )

Train on 7803 samples
Epoch 1/200
7803/7803 - 16s - loss: 7.6431 - accuracy: 0.0263
Epoch 2/200
7803/7803 - 8s - loss: 7.1170 - accuracy: 0.0292
Epoch 3/200
7803/7803 - 9s - loss: 6.9788 - accuracy: 0.0306
Epoch 4/200
7803/7803 - 9s - loss: 6.8561 - accuracy: 0.0386
Epoch 5/200
7803/7803 - 10s - loss: 6.7166 - accuracy: 0.0424
Epoch 6/200
7803/7803 - 10s - loss: 6.5545 - accuracy: 0.0456
Epoch 7/200
7803/7803 - 10s - loss: 6.4542 - accuracy: 0.0500
Epoch 8/200
7803/7803 - 11s - loss: 6.1962 - accuracy: 0.0538
Epoch 9/200
7803/7803 - 13s - loss: 5.9996 - accuracy: 0.0627
Epoch 10/200
7803/7803 - 12s - loss: 5.8182 - accuracy: 0.0625
Epoch 11/200
7803/7803 - 14s - loss: 5.6399 - accuracy: 0.0678
Epoch 12/200
7803/7803 - 13s - loss: 5.4697 - accuracy: 0.0697
Epoch 13/200
7803/7803 - 10s - loss: 5.3026 - accuracy: 0.0779
Epoch 14/200
7803/7803 - 13s - loss: 5.1467 - accuracy: 0.0819
Epoch 15/200
7803/7803 - 10s - loss: 4.9922 - accuracy: 0.0909
Epoch 16/200
7803/7803 - 10s - loss: 4.8487 -

KeyboardInterrupt: 

각 단어의 임베딩 벡터는 10차원을 가지고, 128의 은닉 상태 크기를 가지는 LSTM을 사용합니다. 문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

In [ ]:
def sentence_generation( model, t, current_word, n ): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range( n ): # n번 반복
        encoded = t.texts_to_sequences( [ current_word ] )[ 0 ] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences( [ encoded ], maxlen = 23, padding = 'pre' ) # 데이터에 대한 패딩
        result = model.predict_classes( encoded, verbose = 0 )
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
print( sentence_generation( model, t, 'i', 10 ) )
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

In [ ]:
print( sentence_generation( model, t, 'how', 10 ) )
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성